## Getting Started with Configurable Optimizers ⚡

We designed this library to abstract orthogonal components in the Differentiable NAS Research. But what are these orthogonal components?

Consider, for instance, the [PC-DARTS](https://arxiv.org/abs/1907.05737) method. It has 3 components if we see from high level- a **searchspace** (DARTS), a **sampler** (softmax) to continously sample architectures, and a feature, **partial-connections**. Let use call these one-shot components that we could *plug* 🔌 into searchspace and *play*! 🎮

There are many such components from the past researches in the Differentiable NAS Research. We introduce a high-level API, i.e., **Profile** 🥷 - through which a user can ***configure*** everything about their method.

See this simple example of how we can connect these components together to run PC-DARTS method.

*Before proceeding, please follow the installation instructions in the README.md to install the library to your preffered environment.*

In [ ]:
from confopt.profile import DARTSProfile
from confopt.train import Experiment
from confopt.enums import SearchSpaceType, DatasetType

### Basic Usage ⚒️

We have some pre-configured profiles like 
- ***DARTSProfile*** for softmax sampling,
- ***DrNASProfile*** for sampling from a dirchlet distribution,
- ***GDASProfile*** for sampling from a gumbel-softmax distribution.

There are plenty of options available in these profiles, but for now, we will stick to a minimal version...

Since we are gonna do a softmax sampling, we are gonna use the **DARTSProfile**.

In [ ]:
profile = DARTSProfile(
    searchspace_type=SearchSpaceType.DARTS,
    epochs=3,
    seed=100,
    is_partial_connection=True,
)

Now that we have defined our profile, we want to setup another core fragment of the library - ***Experiment*** 🧪

The ***Experiment*** class defines essentially what the profile is applied on. It takes in `search-space`, `dataset`, `seed`. For the purpose of demo, we will also use an option called `debug_mode` to run it only for testing purpose.    

***Experiment*** in turn, provides api to train the supernetwork and the discrete networks via *train_supernet* and *train_discrete_model* functions.

In [ ]:
experiment = Experiment(
    search_space=SearchSpaceType.DARTS,
    dataset=DatasetType.CIFAR10,
    seed=100,
    debug_mode=True,
    exp_name="simple-example",
)

experiment.train_supernet(profile)

### Advanced Usage ⚒️

Having been familiar with our workflow 📃, we can move to explore some more interesting options we provide in the Profile. 

From plethora of components, let us make a custom profile which will use **GDAS** sampler, that would have [random perturbation](https://arxiv.org/abs/2002.05283), would use [LoRA layers](https://openreview.net/forum?id=YNyTumD0U9&referrer=%5Bthe%20profile%20of%20Frank%20Hutter%5D(%2Fprofile%3Fid%3D~Frank_Hutter1)) with the operations. To top it off, let us also use the [operation-level early stopping](https://proceedings.neurips.cc/paper_files/paper/2023/file/e0bc6dbcbcc957b2aeadb20c39ba7f05-Paper-Conference.pdf), which freezes the operations when they start to overfit.

In the interest to show variety of searchspace we have, we will use **NB201SearchSpace** for this next example. 


*Note:*
- For all options, checkout [BaseProfile](https://github.com/automl/ConfigurableOptimizer/blob/main/src/confopt/profile/base.py).
    
- We support currently 6 searchspaces, [DARTS](https://arxiv.org/abs/1806.09055), [NB201](https://arxiv.org/abs/2001.00326), [NB1SHOT1](https://arxiv.org/abs/2001.10422), [TNB101](https://arxiv.org/abs/2105.11871), [RobustDARTS](https://arxiv.org/abs/1909.09656) and BABYDARTS.
    - Here, the BABYDARTS search space is designed as a toy searchspace for tests.


In [ ]:
from confopt.profile import GDASProfile

profile = GDASProfile(
    searchspace_type=SearchSpaceType.NB201,
    epochs=10,
    perturbation="random",
    lora_rank=1,
    lora_warm_epochs=3,
    oles=True,
    calc_gm_score=True,
    seed=100,
)

profile.configure_oles(frequency=30, threshold=0.4)
profile.configure_lora(
    r=2, lora_alpha=1, lora_dropout=0.1
)  # overwrite previous rank!

As you saw above, you could also configure the methods after initializing the Profile. 
- After `oles` is provided with the Profile, you can configure oles related arguments like *frequency* of steps to measure gm scores, *threshold* to use for early-stop an operation etc.
- Similarly, after lora layers are enabled from Profile, you can configure rank to be lora-related configs, like the *lora alpha*, and *lora dropout* probability for lora layers.

Let us also configure training configs as well, with a batch-size of 96, and use a learning rate of 0.04 for training the supernet and 3e-4 for the architecture.

In [ ]:
profile.configure_trainer(lr=0.04, arch_lr=3e-4, batch_size=96)

All the experiments that we run are logged in a local `log` 🪵 folder, where we save genotypes, model checkpoints, and std logs. These are very helpful to look at after you have finished an experiment run. Additionally, for a better management, we also have an option to log stuff on [**WandB**](https://wandb.ai/) 🪄. 

We also track a lot of metrics which can be helpful to analyse 🔬 experiment like -
- Frequency of operation being picked in genotype per epoch.
- Gradient norms of cells and edges.
- Gradient matching scores for operations.
- alpha values for edges

In [ ]:
# Add as many custom tags/configs here to differentiate runs on WandB
profile.configure_extra(
    project_name="advanced-example",  # Name of the Wandb Project
    run_purpose="test",  # Purpose of the run
)

experiment = Experiment(
    search_space=SearchSpaceType.NB201,
    dataset=DatasetType.CIFAR10,
    seed=100,
    debug_mode=True,
    exp_name="advanced-example",
    log_with_wandb=True,  # enable logging with Weights and Biases
)

experiment.train_supernet(profile)

### Training a Discrete Model 🚅

You have searched for the model. You got an architecture, but how do you test that this architecture is even good 🤔? *We got you covered!*

We have the **DiscreteProfile** that lets you train your model. Every searchspace has their own genotype structure. You should be able to check logs folder to find the best genotype found through the search (we did earlier).

For this example, we would pick the best genotype found by the vanilla DARTS method. Lets take a look at how the genotype looks like -

In [ ]:
from graphviz import Digraph
from io import BytesIO
import PIL.Image
import matplotlib.pyplot as plt
from confopt.searchspace import DARTSGenotype  # noqa: F401


def plot(genotype, genotype_title):
    g = Digraph(
        format="pdf",
        edge_attr=dict(fontsize="20", fontname="times"),
        node_attr=dict(
            style="filled",
            shape="rect",
            align="center",
            fontsize="20",
            height="0.5",
            width="0.5",
            penwidth="2",
            fontname="times",
        ),
        engine="dot",
    )
    g.attr(dpi="600")
    g.body.extend(["rankdir=LR"])

    g.node("c_{k-2}", fillcolor="darkseagreen2")
    g.node("c_{k-1}", fillcolor="darkseagreen2")
    assert len(genotype) % 2 == 0
    steps = len(genotype) // 2

    for i in range(steps):
        g.node(str(i), fillcolor="lightblue")

    for i in range(steps):
        for k in [2 * i, 2 * i + 1]:
            op, j = genotype[k]
            if j == 0:
                u = "c_{k-2}"
            elif j == 1:
                u = "c_{k-1}"
            else:
                u = str(j - 2)
            v = str(i)
            g.edge(u, v, label=op, fillcolor="gray")

    g.node("c_{k}", fillcolor="palegoldenrod")
    for i in range(steps):
        g.edge(str(i), "c_{k}", fillcolor="gray")

    img_bytes = g.pipe(format="png")  # Render as PNG bytes
    img = PIL.Image.open(BytesIO(img_bytes))  # Open with PIL

    # Display in Matplotlib
    plt.figure(figsize=(8, 4))
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.title(genotype_title, fontsize=12, pad=10)
    plt.show()


def plot_genotype(genotype_str):
    try:
        genotype = eval(genotype_str)
    except AttributeError:
        print("{} is not specified in genotypes.py".format(genotype_str))

    plot(genotype.normal, "normal cell")
    plot(genotype.reduce, "reduction cell")


genotype_str = str(
    DARTSGenotype(
        normal=[
            ("sep_conv_3x3", 0),
            ("sep_conv_3x3", 1),
            ("sep_conv_3x3", 0),
            ("sep_conv_3x3", 1),
            ("sep_conv_3x3", 1),
            ("skip_connect", 0),
            ("skip_connect", 0),
            ("dil_conv_3x3", 2),
        ],
        normal_concat=[2, 3, 4, 5],
        reduce=[
            ("max_pool_3x3", 0),
            ("max_pool_3x3", 1),
            ("skip_connect", 2),
            ("max_pool_3x3", 1),
            ("max_pool_3x3", 0),
            ("skip_connect", 2),
            ("skip_connect", 2),
            ("max_pool_3x3", 1),
        ],
        reduce_concat=[2, 3, 4, 5],
    )
)
plot_genotype(genotype_str)

The **`DiscreteProfile`** has an attribute `genotype` that we would set to train the above architecture. With **`DiscreteProfile`**, we can also directly set the trainer config within the initializer.

In [ ]:
from confopt.profile import DiscreteProfile

discrete_profile = DiscreteProfile(
    searchspace_type=SearchSpaceType.DARTS,
    epochs=10,
    seed=100,
    batch_size=96,
    lr=0.03,
)

discrete_profile.configure_extra(
    project_name="Train-Discrete-Model",
    tag="discrete-run",
)
discrete_profile.genotype = genotype_str

Having defined a profile for training architecture, we can now go on to intialize the **`Experiment`**.

In [ ]:
experiment = Experiment(
    search_space=SearchSpaceType.DARTS,
    dataset=DatasetType.CIFAR10,
    seed=100,
    log_with_wandb=True,
    debug_mode=True,
    exp_name="discrete-demo",
)

print("Training model with genotype: ", discrete_profile.genotype)

We expose another api for training discrete model called *`train_discrete_model`* for training a model based on the configurations defined in the `DiscreteProfile`.

In [ ]:
experiment.train_discrete_model(discrete_profile)

Hope you had fun with this tutorial! 👋

🔄 We'd be updating our docs soon with more advanced examples. **Stay tuned!** 🔄